In [ ]:
import os
import time
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer
from chromadb.api.types import Documents, EmbeddingFunction, Embeddings
from typing import List
import sys
sys.path.append("..")
from scripts.config import CHUNKS_PATH
from scripts.config import DATA_PATH
from scripts.chromaDB_handler import ChromaDataManager
import chromadb
import os
from scripts.chromaDB_handler import add_to_chroma_batched



In [ ]:
chromadb.PersistentClient(path=os.path.join(DATA_PATH, "chromadb")).delete_collection(name="textCollection")

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_path = "nomic-ai/nomic-embed-text-v1"
data_manager = ChromaDataManager(model_path=model_path, collection_name='textCollection', data_path=DATA_PATH, device=device)

In [ ]:
# Define columns
import uuid
import json
document_column = 'text'
est_meta_cols = ['chunk_number','file_source']

batch_size = 1000
def process_and_upload_all_jsons(
    input_dir: str):

    for file_name in os.listdir(input_dir):
        if file_name.endswith(".json"):
            file_path = os.path.join(input_dir, file_name)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    json_data = json.load(f)

                
                file_source = file_name.split('.')[0]

                df = pd.DataFrame(json_data)

                add_to_chroma_batched(data_manager, df, doc_col='text', meta_cols=['chunk_number', 'file_source'], batch_size=batch_size, file_id=file_source)

            except Exception as e:
                print(f"Error processing file {file_name}: {e}")
                continue
    return True

In [ ]:
process_and_upload_all_jsons(CHUNKS_PATH)

In [ ]:
# vector search on collection using a query
data_manager.search_vector_store("search: what are the two ways the total cost calculated", n_results=1)